## Write a Spark program to get the below Output based on the below given Input Input :

### Input
```
+--------+-----+------+
|team1   |team2|winner|
+--------+-----+------+
|India   |Aus  |India |
|Srilanka|Aus  |Aus   |
|Srilanka|India|India |
+--------+-----+------+
```
### Output :

```
+--------+-----------+---------+----------+
|   team1|total_match|total_win|total_loss|
+--------+-----------+---------+----------+
|   India|          2|        2|         0|
|Srilanka|          2|        0|         2|
|     Aus|          2|        1|         1|
+--------+-----------+---------+----------+
```

In [0]:
input_data = [
  ('India', 'Aus', 'India'),
  ('Srilanka', 'Aus', 'Aus'),
  ('Srilanka', 'India', 'India'),
]
input_df= spark.createDataFrame(
  input_data, ('team1','team2','winner')
)

input_df.printSchema()

root
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- winner: string (nullable = true)



In [0]:
input_df.show(truncate=False)

+--------+-----+------+
|team1   |team2|winner|
+--------+-----+------+
|India   |Aus  |India |
|Srilanka|Aus  |Aus   |
|Srilanka|India|India |
+--------+-----+------+



In [0]:
from pyspark.sql import Row
def fun(record:Row) :
  if record == None:
    return [Row(team1 = '1', team2 = '2', winner='3')]
  else:
    reverse_record = Row(record['team2'], record['team1'],record['winner'])
    return [record, reverse_record]

In [0]:
input_flat = input_df.rdd.flatMap(fun).toDF(input_df.schema)
input_flat.show(truncate=False)

+--------+--------+------+
|team1   |team2   |winner|
+--------+--------+------+
|India   |Aus     |India |
|Aus     |India   |India |
|Srilanka|Aus     |Aus   |
|Aus     |Srilanka|Aus   |
|Srilanka|India   |India |
|India   |Srilanka|India |
+--------+--------+------+



In [0]:
import pyspark.sql.functions as f
reverse_df = (
  input_df
   .withColumn('team1_new', f.col('team2'))
   .withColumn('team2_new', f.col('team1'))
   .drop('team1','team2')
   .withColumnRenamed('team1_new','team1')
   .withColumnRenamed('team2_new', 'team2')
)
reverse_df.show()

+------+-----+--------+
|winner|team1|   team2|
+------+-----+--------+
| India|  Aus|   India|
|   Aus|  Aus|Srilanka|
| India|India|Srilanka|
+------+-----+--------+



In [0]:
data = input_df.unionByName(reverse_df)
data.show(truncate=False)

+--------+--------+------+
|team1   |team2   |winner|
+--------+--------+------+
|India   |Aus     |India |
|Srilanka|Aus     |Aus   |
|Srilanka|India   |India |
|Aus     |India   |India |
|Aus     |Srilanka|Aus   |
|India   |Srilanka|India |
+--------+--------+------+



In [0]:
data_win_loss = (
  data.withColumn('win_flag', f.when(f.col('team1') == f.col('winner'), f.lit(1)).otherwise(f.lit(0)))
  .withColumn('lose_flag', f.when(f.col('team2') == f.col('winner'), f.lit(1)).otherwise(f.lit(0)))
)
data_win_loss.show(truncate=False)

+--------+--------+------+--------+---------+
|team1   |team2   |winner|win_flag|lose_flag|
+--------+--------+------+--------+---------+
|India   |Aus     |India |1       |0        |
|Srilanka|Aus     |Aus   |0       |1        |
|Srilanka|India   |India |0       |1        |
|Aus     |India   |India |0       |1        |
|Aus     |Srilanka|Aus   |1       |0        |
|India   |Srilanka|India |1       |0        |
+--------+--------+------+--------+---------+



In [0]:
final = (
data_win_loss.groupBy(f.col('team1')).agg(f.count('team2').alias('total_match'), f.sum('win_flag').alias('total_win'), f.sum('lose_flag').alias('total_loss'))
)

final.show()

+--------+-----------+---------+----------+
|   team1|total_match|total_win|total_loss|
+--------+-----------+---------+----------+
|   India|          2|        2|         0|
|Srilanka|          2|        0|         2|
|     Aus|          2|        1|         1|
+--------+-----------+---------+----------+

